<a href="https://colab.research.google.com/github/malonsos/ClimateBert/blob/main/ClimateBert_CEOspolide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 %pip install transformers

In [ ]:
import os
os.getcwd()

In [ ]:
%mkdir Climate_bert

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import re
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# List to store results
results = []

# Iterate through uploaded files
for filename, content in uploaded.items():
    # Decode content to UTF-8
    text = content.decode('utf-8')
    # Split text into paragraphs at each period
    paragraphs = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

    # Load pre-trained model and tokenizer
    model_name = "climatebert/distilroberta-base-climate-detector"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

    # Create text classification pipeline
    pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1)

    # Apply the pipeline to each paragraph
    for i, paragraph in enumerate(paragraphs):
        # Tokenize the paragraph and handle long sequences
        tokens = tokenizer(paragraph, max_length=512, truncation=True, return_overflowing_tokens=True)

        # Iterate over tokenized sequences
        for j in range(len(tokens["input_ids"])):
            input_ids = tokens["input_ids"][j]
            attention_mask = tokens["attention_mask"][j]

            # Convert to string
            truncated_text = tokenizer.decode(input_ids, skip_special_tokens=True)

            # Apply the pipeline to the truncated text
            output = pipe(truncated_text)[0]

            # Append results to the list
            results.append({
                'File Name': filename,
                'Part Number': f"{i + 1}.{j + 1}",
                'Text': truncated_text,
                'Label': output['label'],
                'Score': output['score']
            })

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
csv_file_path = '/content/Climate_bert/classification_results.csv'
df.to_csv(csv_file_path, index=False)

print(f"Results saved to: {csv_file_path}")

In [ ]:
!pip install datasets

In [ ]:
#specificity
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm
from google.colab import files

# Upload your Excel file to the Colab environment
uploaded = files.upload()

# Load the Excel file into a Pandas DataFrame
df = pd.read_excel("classification_results_ejemplo.xlsx")

# Filter rows where label is "yes"
df_yes = df[df['Label'] == 'yes']

# Extract the text from the filtered DataFrame
texts = df_yes['Text'].tolist()

# Initialize model and tokenizer
model_name = "climatebert/distilroberta-base-climate-specificity"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

# Initialize pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1)

# Process the texts through the pipeline
for out in tqdm(pipe(texts, padding=True, truncation=True)):
    print(out)

In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
from google.colab import files

# Upload your Excel file to the Colab environment
uploaded = files.upload()

# Load the Excel file into a Pandas DataFrame
df = pd.read_excel("classification_results_ejemplo.xlsx")

# Filter rows where label is "yes"
df_yes = df[df['Label'] == 'yes']

# Extract the text from the filtered DataFrame
texts = df_yes['Text'].tolist()

# Initialize model and tokenizer
model_name = "climatebert/distilroberta-base-climate-specificity"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

# Initialize pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1)

# Process the texts through the pipeline
results = []
for i, out in tqdm(enumerate(pipe(texts, padding=True, truncation=True))):
    results.append({
        'File Name': df_yes.iloc[i]['File Name'],
        'Part Number': df_yes.iloc[i]['Part Number'],
        'Text': df_yes.iloc[i]['Text'],
        'Label': out['label'],
        'Score': out['score']
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV file
results_df.to_csv("classification_results1.csv", index=False)

# Download the CSV file
files.download("classification_results1.csv")